<a href="https://colab.research.google.com/github/Mandroide/Test_CVPR_AICity_2021_Anomaly_Detection/blob/main/Test_CVPR_AICity_2021_Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# General Setup

Before you start working with this notebook, you need decide if you would like to test the code with the video on the GitHub repository or with a new Video.

If you would like to run a new video (i.e., your own video) , you would need to clone the repository to your own Google Drive. Hence, first you would need to  "mount" your Google Drive in the runtime's Virtual Machine. This will allow you to read and save files from your own Google Drive "CVPR_AICity_2021_Anomaly_Detection" folder. 

To do this, you need to provide the path of your Google Drive for the folder of this assignment and click on the option of “Use_Google_Drive”. Once you provide this information, run this code cell below and follow the instructions. If you don’t click Use_Google_Drive, there is not need to submit a “Working_Directory”


In [1]:
#@title Helper functions
#@markdown During the run of this notebook, we'll use a set of functions to
#@markdown setup the installation of the components
import time
import os
import pathlib
from typing import Optional


def clone_repo(repo: str, depth: int = 1) -> None:
  """Git clone the specified repo."""
  !git clone --depth {depth} --no-tags --recurse-submodules -v {repo}
  #!git clone --no-tags --recurse-submodules -v {repo}

def install_cuda() -> None:
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
  !mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
  !apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
  !add-apt-repository "deb https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/ /"
  !apt-get update
  !apt-get -y install cuda

def install_yolov5() -> None:
  clone_repo("https://github.com/ultralytics/yolov5")
  !pip install -r yolov5/requirements.txt


In [2]:
#@title Available options
#@markdown Choose if you want to mount your Google drive
Use_Google_Drive = True #@param {type:"boolean"}
start_time = time.time()

if Use_Google_Drive:
  from google.colab import drive 
  

  drive.mount("/content/gdrive")

  #@markdown Default drive directory

  Working_Directory = "My Drive/Colab Notebooks/Anomaly_detection/Test_CVPR_AICity_2021_Anomaly_Detection" #@param {type:"string"}
  wd=pathlib.Path("/content/gdrive/"+Working_Directory)
  #os.chdir(wd)

if not os.path.isdir("CVPR_AICity_2021_Anomaly_Detection"):
  clone_repo("https://github.com/Mandroide/CVPR_AICity_2021_Anomaly_Detection.git")

ROOT_PATH = pathlib.Path.cwd()
REPO_PATH = ROOT_PATH/"CVPR_AICity_2021_Anomaly_Detection"
%cd {REPO_PATH}

Mounted at /content/gdrive
Cloning into 'CVPR_AICity_2021_Anomaly_Detection'...
POST git-upload-pack (180 bytes)
POST git-upload-pack (189 bytes)
remote: Enumerating objects: 128, done.
remote: Counting objects: 100% (128/128), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 128 (delta 13), reused 118 (delta 13), pack-reused 0
Receiving objects: 100% (128/128), 50.65 MiB | 28.72 MiB/s, done.
Resolving deltas: 100% (13/13), done.
/content/CVPR_AICity_2021_Anomaly_Detection


# Dual-Modality Vehicle Anomaly Detection via Bilateral Trajectory Tracing (CVPRW 2021)

This uses the [CVPR_AICity_2021_Anomaly_Detection](https://github.com/Mandroide/CVPR_AICity_2021_Anomaly_Detection)

The following image illustrate the process.

![Dual-modality bilateral trajectory tracing mechanism](https://raw.githubusercontent.com/jingyuanchan/CVPR_AICity_2021_Anomaly_Detection/main/figs/abstract.png)

The pipeline of this proposed method that tracking stage provides the exact time and bounding boxes of anomalies in the stationary conditions and localization stage traces back the instants of crashes if possible.


**Resources**

- Paper: [Dual-Modality Vehicle Anomaly Detection via Bilateral Trajectory Tracing](https://openaccess.thecvf.com/content/CVPR2021W/AICity/papers/Chen_Dual-Modality_Vehicle_Anomaly_Detection_via_Bilateral_Trajectory_Tracing_CVPRW_2021_paper.pdf)
- Original repository: [CVPR_AICity_2021_Anomaly_Detection](https://github.com/jingyuanchan/CVPR_AICity_2021_Anomaly_Detection)


In [3]:
#@title Installation

#@markdown ### Requirements
#@markdown - Linux (tested on Ubuntu 16.04.5)
#@markdown - Packages (listed in the [requirements.txt](https://github.com/Mandroide/CVPR_AICity_2021_Anomaly_Detection/blob/main/requirements.txt))
#@markdown - [YOLOv5](https://docs.ultralytics.com/)

# Install YOLOV5
clone_repo("https://github.com/ultralytics/yolov5")
!pip install -r yolov5/requirements.txt

# Install other dependencies:
!pip install -r requirements.txt

Cloning into 'yolov5'...
POST git-upload-pack (180 bytes)
POST git-upload-pack (189 bytes)
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 101 (delta 17), reused 48 (delta 14), pack-reused 0
Receiving objects: 100% (101/101), 1.02 MiB | 23.16 MiB/s, done.
Resolving deltas: 100% (17/17), done.
     |████████████████████████████████| 645kB 8.0MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Run

In [4]:
#@markdown **By default, this comes with a small dataset of videos to detect
#@markdown anomalies**

#@markdown **If you wish to use your own dataset folder, check the option and
#@markdown insert a path**
Custom_dataset = True #@param {type:"boolean"}
Custom_drive = Use_Google_Drive and Custom_dataset

if Custom_drive:
  infer_dir = "/content/gdrive/Shareddrives/UASD Fondocyt Proyecto 911/RA memos/Manuel/ROI_detection/frames/cam_20" #@param {type:"string"}
  #"/content/gdrive/Shareddrives/UASD Fondocyt Proyecto 911/Datasets/AI city challenge/AIC20_track4/test-data"
else:
  infer_dir = "datasets/AIC20_track4/test-data"

infer_dir = pathlib.PurePath(infer_dir)

In [5]:
#@title Background Modeling
#@markdown - Extract background
!python Background_Modeling/ex_bg_mog.py

python3: can't open file 'Background_Modeling/ex_bg_mog.py': [Errno 2] No such file or directory


In [ ]:
#@title Detection
!python yolov5/detect.py --source "{infer_dir}"

detect: weights=yolov5s.pt, source=/content/gdrive/Shareddrives/UASD Fondocyt Proyecto 911/RA memos/Manuel/ROI_detection/frames/cam_20, imgsz=640, conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 dabad57 torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

100% 14.1M/14.1M [00:00<00:00, 53.2MB/s]

Fusing layers... 
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_m

In [7]:
#@title Road Mask Construction
#@markdown - Extract Motion-Based Mask
!python Road_Mask/mask_frame_diff.py
#@markdown - Extract Trajectory-Based Mask
!python Road_Mask/mask_track.py
#@markdown - Mask Fusion
!python Road_Mask/mask_fuse.py

python3: can't open file 'Road_Mask/mask_frame_diff.py': [Errno 2] No such file or directory
python3: can't open file 'Road_Mask/mask_track.py': [Errno 2] No such file or directory
python3: can't open file 'Road_Mask/mask_fuse.py': [Errno 2] No such file or directory


In [8]:
#@title Pixel-level Tracking
#@markdown - Coarse Detect
!python ROI_Backtracking/pixel_track/coarse_ddet/pixel-level_tracking.py
#@markdown - Fuse Similar Results
!python ROI_Backtracking/pixel_track/post_process/similar.py
#@markdown - Filter Suspicious Anomaly Results
!python ROI_Backtracking/pixel_track/post_process/filter.py
#@markdown - Fuse Close Results
!python ROI_Backtracking/pixel_track/post_process/pixel_fuse.py
#@markdown - ROI Backtracking for Pixel-Level
!python ROI_Backtracking/pixel_track/post_process/timeback_pixel.py
#@markdown - Fuse Tracking Results
!python ROI_Backtracking/pixel_track/post_process/sync.py

python3: can't open file 'ROI_Backtracking/pixel_track/coarse_ddet/pixel-level_tracking.py': [Errno 2] No such file or directory
python3: can't open file 'ROI_Backtracking/pixel_track/post_process/similar.py': [Errno 2] No such file or directory
python3: can't open file 'ROI_Backtracking/pixel_track/post_process/filter.py': [Errno 2] No such file or directory
python3: can't open file 'ROI_Backtracking/pixel_track/post_process/pixel_fuse.py': [Errno 2] No such file or directory
python3: can't open file 'ROI_Backtracking/pixel_track/post_process/timeback_pixel.py': [Errno 2] No such file or directory
python3: can't open file 'ROI_Backtracking/pixel_track/post_process/sync.py': [Errno 2] No such file or directory


In [9]:
#@title Step6: Fusion and Backtracking Optimization
#@markdown - Fusion of box_level tracking and pixel-level tracking, and backtracking
!python fusion_code/fusion_backtracking.py

python3: can't open file 'fusion_code/fusion_backtracking.py': [Errno 2] No such file or directory
